In [201]:
import pymysql
import pandas as pd

host = '127.0.0.1'
port = "3306"
user = 'root'
password = 'mysecretpassword'
database = 'gdelt'

conn = pymysql.connect(
    host=host,
    port=int(port),
    user=user,
    passwd=password,
    db=database,
    charset='utf8mb4')

def run(sql):
    df = pd.read_sql_query(sql,conn)
    return df

### 1. View event yang terjadi di indonesia

In [202]:
df = run("SELECT year, actiongeo_fullname, title, content   FROM gdelt_event")
df

,year,actiongeo_fullname,title,content
0,2018,"Bali, Jawa Timur, Indonesia","Goodbye Indonesia, I will miss you",And yet I have never felt more alive. The clog...
1,2018,"Rembang, Jawa Timur, Indonesia","Goodbye Indonesia, I will miss you",And yet I have never felt more alive. The clog...
2,2018,"Rembang, Jawa Timur, Indonesia","Goodbye Indonesia, I will miss you",And yet I have never felt more alive. The clog...
3,2018,"Hanura, Jawa Barat, Indonesia",None,None
4,2018,"Yogyakarta, Yogyakarta, Indonesia",Hazards of overseas employment: OFW cases that...,"Metro Manila (CNN Philippines, February 16) — ..."
5,2018,"Bali, Jawa Timur, Indonesia",FBI asked Indonesian police to impound Jho Low...,JAKARTA (Feb 28): Indonesian police impounded ...
6,2018,"Bali, Jawa Timur, Indonesia",ABC Radio Australia,Scott Morrison will today attend his first int...
7,2018,"Bali, Jawa Timur, Indonesia",ABC Radio Australia,Scott Morrison will today attend his first int...
8,2018,"Hollywood, California, United States",None,None
9,2018,"Bali, Jawa Timur, Indonesia",None,None


### 2. Cleaning data

In [214]:
provinces = ['SUMATERA UTARA', 'SUMATERA BARAT', 'RIAU', 'JAMBI', 'SUMATERA SELATAN', 'BENGKULU', 'LAMPUNG', 'KEPULAUAN BANGKA BELITUNG', 'KEPULAUAN RIAU', 'DKI JAKARTA' , 'JAWA BARAT', 'JAWA TENGAH', 'YOGYAKARTA', 'JAWA TIMUR', 'BANTEN', 'BALI', 'NUSA TENGGARA BARAT', 'NUSA TENGGARA TIMUR', 'KALIMANTAN BARAT', 'KALIMANTAN TENGAH', 'KALIMANTAN SELATAN', 'KALIMANTAN TIMUR', 'KALIMANTAN UTARA', 'SULAWESI UTARA', 'SULAWESI TENGAH', 'SULAWESI SELATAN', 'SULAWESI TENGGARA', 'GORONTALO', 'SULAWESI BARAT', 'MALUKU',  'MALUKU UTARA', 'PAPUA BARAT', 'PAPUA']

def parse_action_geo(v):
    arr_geo = v.split(',')
    return (arr_geo[0], arr_geo[1] if len(arr_geo) > 2 else arr_geo[0], arr_geo[2] if len(arr_geo) > 3 else arr_geo[0])

clean_df = df.copy().drop_duplicates(subset = ['title'])
geo = list(map(parse_action_geo, clean_df['actiongeo_fullname'].values))
# clean_df['city'] = pd.Series([i[0].strip() for i in geo]).values
clean_df['province'] = pd.Series([i[1].strip() for i in geo]).str.upper().values
clean_df = clean_df.drop(columns = ['actiongeo_fullname'])
clean_df = clean_df[clean_df['province'].isin(provinces)]
clean_df = clean_df[['province', 'year', 'title', 'content']]
clean_df = clean_df[clean_df['title'].notnull()]
clean_df

,province,year,title,content
0,JAWA TIMUR,2018,"Goodbye Indonesia, I will miss you",And yet I have never felt more alive. The clog...
4,YOGYAKARTA,2018,Hazards of overseas employment: OFW cases that...,"Metro Manila (CNN Philippines, February 16) — ..."
5,JAWA TIMUR,2018,FBI asked Indonesian police to impound Jho Low...,JAKARTA (Feb 28): Indonesian police impounded ...
6,JAWA TIMUR,2018,ABC Radio Australia,Scott Morrison will today attend his first int...
12,JAWA TIMUR,2018,Polri Dismantles Human Traffickers to Abu Dhab...,"🔊 Listen to this\n\nJAKARTA, NNC - Criminal In..."
14,SUMATERA UTARA,2018,Narcotics Agency Seizes 7 Drug Dealers in Nort...,"Tuesday, 03 April, 2018 | 06:42 WIB\n\nNarcoti..."
16,SUMATERA UTARA,2018,BNN suggests seizure of assets of drug dealers,BNN says death sentence not effective enough.\...
18,JAWA TENGAH,2018,Arts / Final hours mark the end of ‘paradise’,To mark the third anniversary of Sukumaran’s d...
35,JAWA BARAT,2018,France’s Macron weathers bodyguard crisis in p...,Trump envoy urges Russia help free journalist ...
47,JAWA TIMUR,2018,"Pervert, 43, skipped his child-porn trial - af...",Get Daily updates directly to your inbox Subsc...


In [215]:
clean_df.groupby(['province']).size().reset_index(name='counts')

,province,counts
0,JAWA BARAT,1
1,JAWA TENGAH,1
2,JAWA TIMUR,5
3,SULAWESI TENGAH,1
4,SUMATERA UTARA,3
5,YOGYAKARTA,2


In [216]:
clean_df.groupby(['province']).size().reset_index(name='counts').min()

province    JAWA BARAT
counts               1
dtype: object

In [217]:
clean_df.groupby(['province']).size().reset_index(name='counts').max()

province    YOGYAKARTA
counts               5
dtype: object